In [13]:
import sqlite3
import json

from concurrent.futures import ProcessPoolExecutor, as_completed
from copy import deepcopy

import ngram_similarity
import modified_ngram_similarity

from own_infer import chunk

In [14]:
con = sqlite3.connect("meter3.db")
cur = con.cursor()

ngs = ngram_similarity.NgramSimilarity("en")
mngs = modified_ngram_similarity.ModifiedNgramSimilarity("en")

In [15]:
# import cProfile
# import time

# def dostuff():

#     sql_extra_gt = """
#     select
#         t1.id as ida, t2.id as idb
#     from extra as t1
#         cross join extra as t2
#     where
#         t1.id <> t2.id
#     """

#     def nth(it, n):
#         for i, e in enumerate(it):
#             if i % n == 0:
#                 yield e

#     nrows = cur.execute("select count(*) from extra as t1 cross join extra as t2 where t1.id <> t2.id").fetchone()[0]

#     n = 0

#     for ida, idb in nth(cur.execute(sql_extra_gt).fetchall(), int(nrows/2000)):

#         texta = cur.execute(f"select text from extra where id={ida}").fetchone()[0]
#         textb = cur.execute(f"select text from extra where id={idb}").fetchone()[0]
        
#         if len(texta) > 50_000 or len(textb) > 50_000:
#             continue
        
#         st = time.time()

#         scores = ngs.score_texts(texta, textb)
#         mscores = mngs.score_texts(texta, textb, ngram_length=5)

#         print(time.time() - st)

#         if n < 10:
#             n += 1
#         else:
#             break

# cProfile.run('dostuff()', 'meter.profile')

In [16]:
def init_worker(ngs_scorer, mngs_scorer):
    global w_ngs_scorer
    global w_mngs_scorer
    w_ngs_scorer = ngs_scorer
    w_mngs_scorer = mngs_scorer

def score_rows(rows):
    return {(sid, tid): {**w_ngs_scorer.score_texts(a, b), **w_mngs_scorer.score_texts(a, b)} for sid, tid, a, b in rows}

def get_meter_rows():
    for sid, catch, src in cur.execute("SELECT id, catchline, text FROM sources").fetchall():
        for tid, text in cur.execute("SELECT id, text FROM texts WHERE catchline='{}'".format(catch)).fetchall():
            yield (sid, tid, src, text)

In [17]:
# raise KeyboardInterrupt

with ProcessPoolExecutor(max_workers = 24, initializer = init_worker, initargs = (ngram_similarity.NgramSimilarity("en"), modified_ngram_similarity.ModifiedNgramSimilarity("en"))) as executor:
    for batch in chunk(24, chunk(50, get_meter_rows())):
        jobs = []

        for workbundle in batch:
            job = executor.submit(score_rows, workbundle)
            jobs.append(job)
        
        bundleresults = [f.result() for f in as_completed(jobs)]
        for results in bundleresults:
            for (sid, tid), scores in results.items():
                cur.execute(
                    "INSERT OR IGNORE INTO predictions VALUES (?, ?, ?, ?)",
                    (
                        sid,
                        tid,
                        json.dumps(scores, ensure_ascii=False),
                        None
                    )
                )

        con.commit()

In [18]:
# # raise KeyboardInterrupt

# sql_extra_gt = """
# select
#     t1.id as ida, t2.id as idb
# from extra as t1
#     cross join extra as t2
# where
#     t1.id <> t2.id
# """

# def nth(it, n):
#     for i, e in enumerate(it):
#         if i % n == 0:
#             yield e

# nrows = cur.execute("select count(*) from extra as t1 cross join extra as t2 where t1.id <> t2.id").fetchone()[0]

# with ProcessPoolExecutor(max_workers = 22, initializer = init_worker, initargs = (ngram_similarity.NgramSimilarity("en"), modified_ngram_similarity.ModifiedNgramSimilarity("en"))) as executor:
#     for batch in chunk(24, chunk(10, nth(cur.execute(sql_extra_gt).fetchall(), int(nrows/12000)))):
#         jobs = []

#         for workbundle in batch:
#             rows = []
#             for ida, idb in workbundle:
#                 texta = cur.execute(f"select text from extra where id={ida}").fetchone()[0]
#                 textb = cur.execute(f"select text from extra where id={idb}").fetchone()[0]
#                 if not len(texta) > 50_000 and not len(textb) > 50_000:
#                     rows.append((ida, idb, texta, textb))

#             if rows:
#                 job = executor.submit(score_rows, rows)
#                 jobs.append(job)

#         bundleresults = [f.result() for f in as_completed(jobs)]
#         for results in bundleresults:
#             for (ida, idb), scores in results.items():
#                 cur.execute(
#                     "INSERT OR IGNORE INTO extra_predictions VALUES (?, ?, ?, ?)",
#                     (
#                         ida,
#                         idb,
#                         json.dumps(scores, ensure_ascii=False),
#                         None
#                     )
#                 )
#         con.commit()

In [19]:
con.close()
